# Processing California's public payrolls

This notebook processess annual government payroll [data](https://publicpay.ca.gov/Reports/RawExport.aspx) compiled and released annually by the California state controller's office. The data include anonymized salary information for all employees at cities, counties, special districts and state government. 

---

### Load python tools

In [1]:
import pandas as pd
import zipfile
from urllib.request import urlopen 
import pyarrow
import os
import glob
import io
import requests
import matplotlib
import json
import numpy as np
from altair import datum
import altair as alt
alt.renderers.enable('notebook')
import altair_latimes as lat
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')

ThemeRegistry.enable('latimes')

### Download zipped salary tables by year and agency type

In [2]:
os.chdir('/Users/mhustiles/data/data/controller/input/')

In [3]:
formaturl = lambda x: 'https://publicpay.ca.gov/RawExport/' + f'{x[1]}_' + f'{x[0]}' + '.zip'

In [4]:
# metadata = []
# for y in range(2009,2019):
#     for e in ['City', 'County', 'SpecialDistrict', 'StateDepartment']:
# #     for e in ['City', 'County']:
#         metadata.append(dict(entity = e, year = y, url = formaturl((e, y))))

### Extract CSVs from .zip files, and then discard the .zip files

In [5]:
# for m in metadata:
#     !wget '{m['url']}'
#     !unzip \*.zip
#     !rm -f *.zip

---

### Read all the text files, loop and store them in a dataframe

In [6]:
path = '/Users/mhustiles/data/data/controller/input/'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_from_each_file = (pd.read_csv(f, encoding = "ISO-8859-1", low_memory=False, dtype = {'DepartmentOrSubdivision': 'object', 'Year': 'object'}) for f in all_files)
concatenated_df = pd.concat(df_from_each_file, ignore_index=True)

In [7]:
concatenated_df.head()

,Year,EmployerType,EmployerName,DepartmentOrSubdivision,Position,ElectedOfficial,Judicial,OtherPositions,MinPositionSalary,MaxPositionSalary,...,HealthDentalVision,TotalRetirementAndHealthContribution,PensionFormula,EmployerURL,EmployerPopulation,LastUpdatedDate,EmployerCounty,SpecialDistrictActivities,IncludesUnfundedLiability,SpecialDistrictType
0,2010,County,Alameda,Ac Central Collections,Accountant-Auditor,NaN,NaN,NaN,53566.0,58324.0,...,1088.0,1088.0,2%@62,www.acgov.org/hrs,1517756.0,06/06/2016,Alameda,NaN,NaN,NaN
1,2010,County,Alameda,Ac Central Collections,Accounting Technician,NaN,NaN,NaN,45427.0,55224.0,...,12375.0,14031.0,2%@57,www.acgov.org/hrs,1517756.0,06/06/2016,Alameda,NaN,NaN,NaN
2,2010,County,Alameda,Ac Central Collections,Admin Specialist II,NaN,NaN,NaN,56451.0,75608.0,...,4718.0,5486.0,2%@57,www.acgov.org/hrs,1517756.0,06/06/2016,Alameda,NaN,NaN,NaN
3,2010,County,Alameda,Ac Central Collections,Auditor Associate I,NaN,NaN,NaN,40033.0,47346.0,...,8710.0,8710.0,2%@62,www.acgov.org/hrs,1517756.0,06/06/2016,Alameda,NaN,NaN,NaN
4,2010,County,Alameda,Ac Central Collections,Auditor Associate I,NaN,NaN,NaN,40033.0,47346.0,...,4916.0,4916.0,2%@62,www.acgov.org/hrs,1517756.0,06/06/2016,Alameda,NaN,NaN,NaN


### Trim the dataframe to the columns we need

In [8]:
payroll = pd.DataFrame(concatenated_df[['Year','EmployerType','EmployerPopulation','EmployerName','DepartmentOrSubdivision',\
                 'Position','OvertimePay','TotalWages', 'TotalRetirementAndHealthContribution', 'EmployerCounty']])

In [9]:
payroll.head()

,Year,EmployerType,EmployerPopulation,EmployerName,DepartmentOrSubdivision,Position,OvertimePay,TotalWages,TotalRetirementAndHealthContribution,EmployerCounty
0,2010,County,1517756.0,Alameda,Ac Central Collections,Accountant-Auditor,NaN,4120.0,1088.0,Alameda
1,2010,County,1517756.0,Alameda,Ac Central Collections,Accounting Technician,NaN,54438.0,14031.0,Alameda
2,2010,County,1517756.0,Alameda,Ac Central Collections,Admin Specialist II,NaN,25376.0,5486.0,Alameda
3,2010,County,1517756.0,Alameda,Ac Central Collections,Auditor Associate I,NaN,31225.0,8710.0,Alameda
4,2010,County,1517756.0,Alameda,Ac Central Collections,Auditor Associate I,NaN,27677.0,4916.0,Alameda


### Not everyone reports (or pays) overtime

In [10]:
payroll['OvertimePay'].fillna(0, inplace = True)
payroll['DepartmentOrSubdivision'].fillna('NOT LISTED', inplace = True)
payroll['EmployerPopulation'].fillna(0, inplace = True)

In [11]:
payroll.head()

,Year,EmployerType,EmployerPopulation,EmployerName,DepartmentOrSubdivision,Position,OvertimePay,TotalWages,TotalRetirementAndHealthContribution,EmployerCounty
0,2010,County,1517756.0,Alameda,Ac Central Collections,Accountant-Auditor,0.0,4120.0,1088.0,Alameda
1,2010,County,1517756.0,Alameda,Ac Central Collections,Accounting Technician,0.0,54438.0,14031.0,Alameda
2,2010,County,1517756.0,Alameda,Ac Central Collections,Admin Specialist II,0.0,25376.0,5486.0,Alameda
3,2010,County,1517756.0,Alameda,Ac Central Collections,Auditor Associate I,0.0,31225.0,8710.0,Alameda
4,2010,County,1517756.0,Alameda,Ac Central Collections,Auditor Associate I,0.0,27677.0,4916.0,Alameda


### Clean up column headers

In [12]:
payroll.columns = payroll.columns.str.strip().str.lower().str.replace(' ', '_')\
                    .str.replace('(', '').str.replace(')', '').str.replace('-','_')

In [13]:
payroll.head()

,year,employertype,employerpopulation,employername,departmentorsubdivision,position,overtimepay,totalwages,totalretirementandhealthcontribution,employercounty
0,2010,County,1517756.0,Alameda,Ac Central Collections,Accountant-Auditor,0.0,4120.0,1088.0,Alameda
1,2010,County,1517756.0,Alameda,Ac Central Collections,Accounting Technician,0.0,54438.0,14031.0,Alameda
2,2010,County,1517756.0,Alameda,Ac Central Collections,Admin Specialist II,0.0,25376.0,5486.0,Alameda
3,2010,County,1517756.0,Alameda,Ac Central Collections,Auditor Associate I,0.0,31225.0,8710.0,Alameda
4,2010,County,1517756.0,Alameda,Ac Central Collections,Auditor Associate I,0.0,27677.0,4916.0,Alameda


In [14]:
payroll.rename(columns = {
'employertype':'type',
'employerpopulation':'population',
'employername':'employer',
'departmentorsubdivision':'department',
'overtimepay':'overtime',
'totalretirementandhealthcontribution':'benefits',
'totalwages':'wages',
 }, inplace = True)

### Uppercase everything because their title casing across hundreds of agencies is janky

In [15]:
payroll = payroll.apply(lambda x: x.astype(str).str.upper())

### How do the dataframes look? 

In [16]:
payroll.head()

,year,type,population,employer,department,position,overtime,wages,benefits,employercounty
0,2010,COUNTY,1517756.0,ALAMEDA,AC CENTRAL COLLECTIONS,ACCOUNTANT-AUDITOR,0.0,4120.0,1088.0,ALAMEDA
1,2010,COUNTY,1517756.0,ALAMEDA,AC CENTRAL COLLECTIONS,ACCOUNTING TECHNICIAN,0.0,54438.0,14031.0,ALAMEDA
2,2010,COUNTY,1517756.0,ALAMEDA,AC CENTRAL COLLECTIONS,ADMIN SPECIALIST II,0.0,25376.0,5486.0,ALAMEDA
3,2010,COUNTY,1517756.0,ALAMEDA,AC CENTRAL COLLECTIONS,AUDITOR ASSOCIATE I,0.0,31225.0,8710.0,ALAMEDA
4,2010,COUNTY,1517756.0,ALAMEDA,AC CENTRAL COLLECTIONS,AUDITOR ASSOCIATE I,0.0,27677.0,4916.0,ALAMEDA


### How many records do we have here?

In [17]:
# How many records?
len(payroll)

10826479

---

### Export to a lightweight format

In [18]:
payroll.reset_index().to_feather('/Users/mhustiles/data/data/controller/output/payroll.feather')

In [19]:
payroll.reset_index().to_csv('/Users/mhustiles/data/data/controller/output/payroll.csv')

---

Data source: https://publicpay.ca.gov/Reports/RawExport.aspx